In [1]:
import pandas as pd
import numpy as np

In [2]:
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    
    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    # Length of lists (as real numbers)
    T = float(len(actual_lst))
    
    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

In [3]:
df=pd.read_excel("Forecasts.xlsx")

In [4]:
df.head()

,Unnamed: 0,Date,Actuals,Forecast without PH,Forecast with PH,HAR,HARX
0,5,2017-10-02,0.004688,0.003330,0.003515,0.004635,0.004729
1,6,2017-10-03,0.004585,0.003915,0.003379,0.005038,0.004458
2,7,2017-10-04,0.003926,0.003561,0.003277,0.005107,0.004488
3,8,2017-10-05,0.002841,0.003787,0.003281,0.005119,0.004484
4,9,2017-10-06,0.002729,0.003504,0.003275,0.005121,0.004485


In [5]:
dm_test(df["Actuals"], df["Forecast without PH"], df["Forecast with PH"], h = 1, crit="MSE")

dm_return(DM=4.078089553903216, p_value=4.868249982543794e-05)

In [6]:
dm_test(df["Actuals"], df["HAR"], df["Forecast with PH"], h = 1, crit="MSE")

dm_return(DM=4.526143287596833, p_value=6.661152048176195e-06)

In [7]:
dm_test(df["Actuals"], df["HARX"], df["Forecast with PH"], h = 1, crit="MSE")

dm_return(DM=3.8699099239256123, p_value=0.00011528285759031771)

In [8]:
dm_test(df["Actuals"], df["HARX"], df["HAR"], h = 1, crit="MSE")

dm_return(DM=-2.0626829869640555, p_value=0.03937729995169064)

In [9]:
dm_test(df["Actuals"], df["HARX"], df["Forecast without PH"], h = 1, crit="MSE")

dm_return(DM=-0.1656840129287941, p_value=0.8684361008224153)

In [10]:
dm_test(df["Actuals"], df["HAR"], df["Forecast without PH"], h = 1, crit="MSE")

dm_return(DM=1.1859405326765688, p_value=0.2359021203747066)

In [11]:
!pip install arch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.4/916.4 kB 34.8 MB/s eta 0:00:00


In [12]:
df1=df
df1["Forecast without PH"]=(df1["Actuals"]-df1["Forecast without PH"])**2
df1["Forecast with PH"]=(df1["Actuals"]-df1["Forecast with PH"])**2
df1["HAR"]=(df1["Actuals"]-df1["HAR"])**2
df1["HARX"]=(df1["Actuals"]-df1["HARX"])**2

In [13]:
df1.head()

,Unnamed: 0,Date,Actuals,Forecast without PH,Forecast with PH,HAR,HARX
0,5,2017-10-02,0.004688,1.842719e-06,1.375145e-06,2.825006e-09,1.674918e-09
1,6,2017-10-03,0.004585,4.490990e-07,1.453431e-06,2.052213e-07,1.624057e-08
2,7,2017-10-04,0.003926,1.333280e-07,4.219384e-07,1.395063e-06,3.152341e-07
3,8,2017-10-05,0.002841,8.955600e-07,1.937511e-07,5.190627e-06,2.700786e-06
4,9,2017-10-06,0.002729,5.999904e-07,2.978133e-07,5.723154e-06,3.082665e-06


In [14]:
from arch.bootstrap import MCS

In [15]:
losses=df.drop(columns=["Date", "Actuals"])

In [16]:
mcs = MCS(losses, size=0.1, method="R", block_size=1109)
mcs.compute()
print("MCS P-values")
print(mcs.pvalues)
print("Included")
included = mcs.included
print(included)
print("Excluded")
excluded = mcs.excluded
print(excluded)

MCS P-values
                     Pvalue
Model name                 
Unnamed: 0            0.000
HAR                   0.000
HARX                  0.009
Forecast without PH   0.012
Forecast with PH      1.000
Included
['Forecast with PH']
Excluded
['Forecast without PH', 'HAR', 'HARX', 'Unnamed: 0']
